# 作業
請使用不同的資料集，並使用 hyper-parameter search 的方式，看能不能找出最佳的超參數組合

In [39]:
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier
import numpy as np

In [10]:
#讀入手寫辨識資料
digits = datasets.load_digits()

# 切分訓練集/測試集
x_train, x_test, y_train, y_test = train_test_split(digits.data, digits.target, test_size=0.25, random_state=4)

# 建立模型
GBC = GradientBoostingClassifier(random_state=7)

# 訓練模型
GBC.fit(x_train, y_train)

# 預測測試集
GBC_y_pred = GBC.predict(x_test)

In [11]:
GBC_acc = metrics.accuracy_score(y_test, GBC_y_pred)
print("GBC Acuuracy: ", GBC_acc)

GBC Acuuracy:  0.9666666666666667


In [33]:
# 設定要訓練的超參數組合
n_estimators = [100, 500, 1000]
max_depth = [5, 10, 15, 20]
param_grid = dict(n_estimators=n_estimators, max_depth=max_depth)

## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
grid_search = GridSearchCV(GBC, param_grid, scoring="accuracy", n_jobs=-1, verbose=1)

# 開始搜尋最佳參數
grid_result = grid_search.fit(x_train, y_train)

# 預設會跑 3-fold cross-validadtion，總共 12 種參數組合，總共要 train 36 次模型

Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:  1.7min finished


In [34]:
# 印出最佳結果與最佳參數
print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best Accuracy: 0.930958 using {'max_depth': 5, 'n_estimators': 100}


In [35]:
grid_result.best_params_

{'max_depth': 5, 'n_estimators': 100}

In [36]:
# 使用最佳參數重新建立模型
GBC_bestparam = GradientBoostingClassifier(max_depth=grid_result.best_params_['max_depth'],
                                           n_estimators=grid_result.best_params_['n_estimators'])
# 訓練模型
GBC_bestparam.fit(x_train, y_train)

# 預測測試集
y_pred = GBC_bestparam.predict(x_test)

In [37]:
# 調整參數後沒有比原來設定的低
print(metrics.accuracy_score(y_test, y_pred))

0.96


In [53]:
#利用隨機搜尋法，尋找最佳參數
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 50, stop = 2000, num = 5)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 5)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 20, 50]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4, 10, 20, 50]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,}
print(random_grid)

{'n_estimators': [50, 537, 1025, 1512, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 35, 60, 85, 110, None], 'min_samples_split': [2, 5, 10, 20, 50], 'min_samples_leaf': [1, 2, 4, 10, 20, 50]}


In [54]:
GBC_random = RandomizedSearchCV(estimator = GBC, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=7, n_jobs = -1)
GBC_random.fit(x_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  4.7min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 14.0min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 26.3min finished


RandomizedSearchCV(cv=3, error_score='raise',
          estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=7, subsample=1.0, verbose=0,
              warm_start=False),
          fit_params=None, iid=True, n_iter=100, n_jobs=-1,
          param_distributions={'n_estimators': [50, 537, 1025, 1512, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 35, 60, 85, 110, None], 'min_samples_split': [2, 5, 10, 20, 50], 'min_samples_leaf': [1, 2, 4, 10, 20, 50]},
          pre_dispatch='2*n_jobs', random_state=7, refit=True,
          return_train_score='warn', scoring=None, verbose=2)

In [55]:
GBC_random.best_params_

{'n_estimators': 537,
 'min_samples_split': 5,
 'min_samples_leaf': 20,
 'max_features': 'sqrt',
 'max_depth': 35}

In [57]:
#最佳模型，提升了約2%準確率
GBC_best_random = GBC_random.best_estimator_

# 使用最佳參數重新建立模型
y_pred = GBC_best_random.predict(x_test)

# 調整參數後沒有比原來設定的低
print(metrics.accuracy_score(y_test, y_pred))

0.9888888888888889
